In [1]:

# Colab 에서 돌린 것
# Mecab 사용
# Sentence Split을 Mecab.morphs 로 진행
# 결과 좋지 않더라

In [2]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.


In [3]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [4]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 49.4 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-12-15 04:33:48--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=TbNRrvZIgI0p4G%2FOx5u2I%2FjbKfI

In [5]:
cd ..

/content


In [57]:
lang1 = "ko"
lang2 = "jeju"
lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
    read().strip().split('\n')

from konlpy.tag import Mecab
mecab = Mecab()
mecab.morphs(lines[3])

['응', ',', '그러니까', '.', '응', ',', '거', '니까', '.']

In [62]:
lines[:5]

['판관했던 거 ?\t판관헤난 거 ?',
 '우리 그냥 맨손에 맨손에 김도 매고 , 김도 베고 . 그렇게 했었어 , 옛날은 .\t우리 그냥 맨손에 맨손에 검질도 메고 , 검질도 비고 . 경헤난 , 옛날은 .',
 '예 .\t에 .',
 '응 , 그러니까 .\t응 , 거니까 .',
 '운영해 오다가 .\t운영헤 오다가 .']

In [17]:
from konlpy.tag import Mecab
mecab = Mecab()

In [18]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
SOS_token = 0
EOS_token = 1

# 단순히 word 를 index 형태로만 만들고 비교하는 그런 모델이라서 성능이 많이 떨어지는 것 같고
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # SOS 와 EOS 포함

    def addSentence(self, sentence):
        for word in mecab.morphs(sentence) : # 형태소 분석해서 넣으면, split 보다 낫지 않을까?
            self.addWord(word)

        # for word in sentence.split(' '):
        #     self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [20]:
# 예제에서는 영어 <-> 불어 였는데
# 표준어 <-> 제주어라서 전처리를 다르게 처리했고
def normalizeString(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^ㄱ-ㅎ가-힣.!?]+", r" ", s)
    return s

In [63]:
# 데이터 형식은 여기에서 사용했던 양식을
# 저희 데이터를 수정해서 이 모델에서 사용했던 형식으로 바꾸어서 그대로 사용했습니다
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # 파일을 읽고 줄로 분리
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    # 임의로 추가 확인용 test
    lines = lines[:30000]

    # 모든 줄을 쌍으로 분리하고 정규화
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # 쌍을 뒤집고, Lang 인스턴스 생성
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [64]:
MAX_LENGTH = 64


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [65]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('ko', 'jeju', True)
print(random.choice(pairs))

Reading lines...
Read 30000 sentence pairs
Trimmed to 29863 sentence pairs
Counting words...
Counted words:
jeju 12405
ko 11123
['예 .', '예 .']


In [66]:
pairs[:5]

[['판관헤난 거 ?', '판관했던 거 ?'],
 ['우리 그냥 맨손에 맨손에 검질도 메고 검질도 비고 . 경헤난 옛날은 .',
  '우리 그냥 맨손에 맨손에 김도 매고 김도 베고 . 그렇게 했었어 옛날은 .'],
 ['에 .', '예 .'],
 ['응 거니까 .', '응 그러니까 .'],
 ['운영헤 오다가 .', '운영해 오다가 .']]

In [67]:
# 이 모델은 pytorch로 사용해서
# 막 fair-seq 처럼 transformer 이런 것 사용하지는 않았고
# 이런식으로 흘러가는데 저도 잘 몰라여!
# 근데 설명은 잘 나와있씁니다
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [68]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [69]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [70]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in mecab.morphs(sentence)]
#    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [71]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [72]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [78]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
      try : 
          training_pair = training_pairs[iter - 1]
          input_tensor = training_pair[0]
          target_tensor = training_pair[1]
          # print(training_pair, "input_tensor:", input_tensor, target_tensor)
          loss = train(input_tensor, target_tensor, encoder,
                      decoder, encoder_optimizer, decoder_optimizer, criterion)
          print_loss_total += loss
          plot_loss_total += loss

          if iter % print_every == 0:
              print_loss_avg = print_loss_total / print_every
              print_loss_total = 0
              print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                          iter, iter / n_iters * 100, print_loss_avg))

          if iter % plot_every == 0:
              plot_loss_avg = plot_loss_total / plot_every
              plot_losses.append(plot_loss_avg)
              plot_loss_total = 0
      except : 
        pass
    showPlot(plot_losses)

In [79]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [80]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [81]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [82]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# 예시에서는 75,000 이었음
# 표준어-제주어 300000만 경우를 기존에 돌렸을 때에도 성능이 좋지 못했음
trainIters(encoder1, attn_decoder1, 50000, print_every=5000)

# 서버에 비해서 학습 속도가 오래 걸리기 때문에
# 인고의 시간이 필요하다.
# 50000 기준으로 45분 정도 예상.
# colab 가용시간 문제로 인해서, 데이터 전처리 후에 -> 서버에서 돌리는 방안이 필요할 수도 있을 듯

4m 15s (- 38m 19s) (5000 10%) 3.8151
8m 54s (- 35m 36s) (10000 20%) 3.7417
13m 25s (- 31m 19s) (15000 30%) 3.4356
17m 58s (- 26m 57s) (20000 40%) 3.2960
22m 35s (- 22m 35s) (25000 50%) 3.1941
27m 21s (- 18m 14s) (30000 60%) 3.0767
32m 9s (- 13m 46s) (35000 70%) 3.0055
36m 59s (- 9m 14s) (40000 80%) 2.9730
41m 51s (- 4m 39s) (45000 90%) 2.8908
46m 32s (- 0m 0s) (50000 100%) 2.8509


In [83]:
evaluateRandomly(encoder1, attn_decoder1)

> 으 . 이디 이디 . 젖지도 말렌 허주만은 똑 여기 .
= 으 . 여기 여기 . 젖지도 말라고 하지만 꼭 여기 .
< 으 . 여기 . . 말 말 은 말 은 말 은 은 <EOS>

> 그걸로 못치기 헤근에 . 무사 안 뒈멘 ?
= 그걸로 못치기 해서 . 왜 안되고 있니 ?
< 그걸로 치 기 하 면 . <EOS>

> 거 친제 친제 .
= 그거 친제 친제 .
< 그거 거 . . <EOS>

> 음 소설 때 .
= 음 소설 때 .
< 음 때 . . . . <EOS>

> 기자 뱀 물리민 에염더레 강 오줌 싸민 멘도롱헌 오줌 싸멍 라낫 주게 게도 막 좋지 안 허여 .
= 그저 뱀 물리면 옆으로 가서 오줌 싸면 따뜻한 오줌 싸면서 발랐었지 . 그래도 아주 좋지 않아 .
< 그것 도 해서 해서 면 면 면 지 면 도 하 먹 지 면 지 <EOS>

> 아 그때 월경 와실 때 월경을 막는 따로 뭐 이렇게 천이 엇엇수과 ?
= 아 그때 월경 왔을 때 월경을 막는 따로 뭐 이렇게 천이 없었습니까 ?
< 아 그때 그때 그때 사람 그때 는 사람 었 었 었 습니까 ? <EOS>

> 간 ?
= 간 ?
< 오 ? <EOS>

> 지들커도 옛날에 허젠 허민 힘들어 낫주예 ?
= 땔감도 옛날에 하려고 하면 힘들었었지요 ?
< 옛날 옛날 은 옛날 하 하 면 었 <EOS>

> 어 . 가레기 그 물레에 그 허면 지금 으면 뭘로 알아야 허면 그 저 이런 뭐 잇잖아 . 공장에 가면 방에질 허는 디 .
= 어 . 가락 그 물레에 그 하면 지금 같으면 뭘로 알아야 하면 그 저 이런 뭐 있잖아 . 공장에 가면 방아질 하는 데 .
< 어 . 하 면 그 그 하 면 면 면 하 면 가 . . 면 . . <EOS>

>  새기 그다음 .
= 달걀 그다음 .
< 그 다음 . . <EOS>



In [84]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 200000, print_every=5000)

evaluateRandomly(encoder1, attn_decoder1)

4m 59s (- 194m 29s) (5000 2%) 3.7808
9m 30s (- 180m 35s) (10000 5%) 3.6960
14m 12s (- 175m 9s) (15000 7%) 3.4693
18m 52s (- 169m 54s) (20000 10%) 3.2583
23m 28s (- 164m 19s) (25000 12%) 3.1392
28m 16s (- 160m 15s) (30000 15%) 3.0588
33m 2s (- 155m 46s) (35000 17%) 2.9751
37m 50s (- 151m 23s) (40000 20%) 2.9076
42m 37s (- 146m 49s) (45000 22%) 2.8774
47m 21s (- 142m 4s) (50000 25%) 2.8274
52m 1s (- 137m 9s) (55000 27%) 2.8493
56m 44s (- 132m 24s) (60000 30%) 2.8690
61m 26s (- 127m 37s) (65000 32%) 2.9885
66m 11s (- 122m 56s) (70000 35%) 2.9417
70m 47s (- 117m 59s) (75000 37%) 3.0520
75m 12s (- 112m 49s) (80000 40%) 3.2859
79m 42s (- 107m 50s) (85000 42%) 4.1314
84m 3s (- 102m 44s) (90000 45%) 4.6057
88m 29s (- 97m 48s) (95000 47%) 4.5687
97m 13s (- 87m 58s) (105000 52%) 9.0772
101m 43s (- 83m 13s) (110000 55%) 4.7000
106m 12s (- 78m 29s) (115000 57%) 4.5910
110m 33s (- 73m 42s) (120000 60%) 4.4130
114m 58s (- 68m 59s) (125000 62%) 4.3932
119m 31s (- 64m 21s) (130000 65%) 4.4889
123m 58s

In [52]:
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)


In [53]:
encoder1

EncoderRNN(
  (embedding): Embedding(25010, 256)
  (gru): GRU(256, 256)
)

In [54]:
attn_decoder1

AttnDecoderRNN(
  (embedding): Embedding(21452, 256)
  (attn): Linear(in_features=512, out_features=64, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=21452, bias=True)
)

In [36]:
output_lang